In [13]:
import os
import re
import sys
import glob

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from Gaugi import load as gload

# add gaugi by hand
sys.path.append('../../.__python__/')
from kolmov import table_info, dump_all_train_history, dump_train_history

In [14]:
base_path   = '/volume/'
output_path = os.path.join(base_path, 'models_history_v1_nohad')
#task_name   = 'v1_task/user.mverissi.task.data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.GRL_v97_*/*'
task_name   = 'v1_task_nohad/user.mverissi.task_nohad.data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.GRL_v97_*/*'

task_full_name = os.path.join(base_path, task_name)

In [15]:
# check the paths
print(output_path)
print(task_full_name)

/volume/models_history_v1_nohad
/volume/v1_task_nohad/user.mverissi.task_nohad.data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.GRL_v97_*/*


In [16]:
# wanted information
tuned_info = {
    # validation
    'max_sp_val'     : [],
    'max_sp_pd_val'  : [],
    'max_sp_fa_val'  : [],
    'auc_val'        : [],
    # operation
    'max_sp_op'     : [],
    'max_sp_pd_op'  : [],
    'max_sp_fa_op'  : [],
    'auc_op'        : [],
}

In [18]:
# init the table_info
test = table_info(task_full_name, tuned_info, tag='v1_nohad')

There are 900 files for this task...


In [19]:
# fill table
test.fill_table()

Filling the table... 
End of fill step, a pandas DataFrame was created...


In [20]:
cv_df = test.get_pandas_table()
cv_df.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,total_sgn,total_bkg,max_sp_val,max_sp_pd_val,max_sp_fa_val,auc_val,max_sp_op,max_sp_pd_op,max_sp_fa_op,auc_op
0,v1_nohad,2,3,8,0,40,tunedDiscr.jobID_0044.pic.gz,3614,250734,0.953816,0.983425,0.075340,0.984372,0.944448,0.959325,0.070314,0.979096
1,v1_nohad,2,3,8,0,41,tunedDiscr.jobID_0044.pic.gz,3614,250734,0.955034,0.977901,0.067563,0.982657,0.944888,0.953791,0.063972,0.977564
2,v1_nohad,2,3,8,0,42,tunedDiscr.jobID_0044.pic.gz,3614,250734,0.953634,0.983425,0.075699,0.983870,0.944224,0.959325,0.070756,0.978270
3,v1_nohad,2,3,8,0,43,tunedDiscr.jobID_0044.pic.gz,3614,250734,0.953548,0.980663,0.073186,0.981857,0.943464,0.960985,0.073895,0.976147
4,v1_nohad,2,3,8,0,44,tunedDiscr.jobID_0044.pic.gz,3614,250734,0.953002,0.980663,0.074263,0.983747,0.943320,0.958495,0.071733,0.978233


In [21]:
# filter the DataFrame in order to get only 1 init.
# ordened by max_sp_val
cv_df = test.filter_inits('max_sp_val')

In [22]:
# only 1 init in each sort
len(cv_df)

1650

In [23]:
# save the filterd DataFrame
test.dump_table(cv_df, base_path, 'cv_table_v1_nohad')

In [24]:
cv_df.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,total_sgn,total_bkg,max_sp_val,max_sp_pd_val,max_sp_fa_val,auc_val,max_sp_op,max_sp_pd_op,max_sp_fa_op,auc_op
9008,v1_nohad,0,0,0,0,8,tunedDiscr.jobID_0000.pic.gz,28455,205005,0.941784,0.968377,0.084439,0.978992,0.938199,0.956247,0.079676,0.978314
8573,v1_nohad,0,0,0,1,43,tunedDiscr.jobID_0004.pic.gz,28455,205005,0.938560,0.960295,0.082927,0.976318,0.937698,0.956001,0.080427,0.978230
10843,v1_nohad,0,0,0,2,23,tunedDiscr.jobID_0002.pic.gz,28455,205005,0.934502,0.941321,0.072293,0.975314,0.937448,0.954630,0.079579,0.977537
10868,v1_nohad,0,0,0,3,28,tunedDiscr.jobID_0002.pic.gz,28455,205005,0.937192,0.956781,0.082195,0.978030,0.937805,0.957301,0.081491,0.978082
10880,v1_nohad,0,0,0,4,20,tunedDiscr.jobID_0002.pic.gz,28455,205005,0.938182,0.957836,0.081268,0.976505,0.938043,0.954490,0.078262,0.978116


In [27]:
# save all the train evolution into json files
task_p = '/volume/v1_task_nohad/user.mverissi.task_nohad.data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.GRL_v97_et%i_eta%i.v1'
dump_all_train_history(cv_df, task_p, output_path)